In [1]:
### TEST
import os
import sys
from pathlib import Path

nb_dir = Path(os.getcwd())

# 프로젝트 루트: notebooks/Jang -> notebooks -> project_root
project_root = nb_dir.parents[1]  # /data/ephemeral/pro-nlp-generationfornlp-nlp-13

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("project_root:", project_root)
print("sys.path[0]:", sys.path[0])

project_root: /data/ephemeral/pro-nlp-generationfornlp-nlp-13
sys.path[0]: /data/ephemeral/pro-nlp-generationfornlp-nlp-13


In [2]:
from transformers import AutoTokenizer


from src.prompt.prompt_registry import PromptRegistry
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.data.data_loader import DataConfig, make_train_valid_dataset
from src.data.tokenizer_wrapper import TokenizerConfig

In [3]:
policy = {
    "system": {4: "v1", 5: "v1"},
    "user":   {4: "v1", 5: "v1"},
}

prompt_cfg = PromptConfig(
    policy=policy,
    mode="train",
    verbose=True
)

data_cfg = DataConfig(
    train_path=project_root / "data" / "train.csv",
    valid_ratio=0.1,
    seed=42,
    do_split=True,
)

tokenize_cfg_train = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=False,
)

tokenize_cfg_gen = TokenizerConfig(
    max_length=2048,
    truncation=True,
    padding=False,
    add_generation_prompt=True,
)

MODEL_NAME = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

In [4]:
ds = make_train_valid_dataset(
    data_cfg=data_cfg,
    prompt_cfg=prompt_cfg,
    tokenize_cfg_train=tokenize_cfg_train,
    tokenize_cfg_gen=tokenize_cfg_gen,
    tokenizer=tokenizer,
)


template loading 완료: system=2, user_4=2, user_5=2
template loading 완료: system=2, user_4=2, user_5=2


Build train messages:   0%|          | 0/1827 [00:00<?, ? examples/s]

Serialize train to text:   0%|          | 0/1827 [00:00<?, ? examples/s]

Build valid messages (teacher forcing):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid to text:   0%|          | 0/204 [00:00<?, ? examples/s]

Build valid_gen messages (prompt only):   0%|          | 0/204 [00:00<?, ? examples/s]

Serialize valid_gen to text (+meta):   0%|          | 0/204 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'text'],
        num_rows: 1827
    })
    validation: Dataset({
        features: ['id', 'label', 'text'],
        num_rows: 204
    })
    validation_gen: Dataset({
        features: ['id', 'paragraph', 'question_plus', 'question', 'choices', 'answer', 'choices_len', 'text'],
        num_rows: 204
    })
})

In [6]:
ds['train'][0]

{'id': 'generation-for-nlp-2661',
 'label': 3,
 'text': '<|im_start|>system\n당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>\n<|im_start|>user\n### 지문\n“올해 미국 경제에는 태양과 달과 별이 한 줄로 서는 행운이 다가오고 있다.”제이미 다이먼 JP모간체이스 최고경영자(CEO·사진)는 지난주 실적발표 후 투자자들과의 화상 회의에서 이렇게 말했다. 평소 미국 경제를 “조심스럽게 낙관한다”고 말해온 다이먼 CEO는 이날 ‘조심스러운’이라는 단어를 사용하지 않았다. 그는 “실제로 경제 전망이 낙관적이기 때문에 ‘낙관적’이라고 말하는 것”이라며 “대기업, 중소기업, 주식시장, 주택시장 등 어느 한 곳에서도 취약한 부분을 찾기 어렵다”고 덧붙였다.다이먼 CEO뿐 아니다. 월스트리트 대형 은행의 최고 경영진도 잇따라 미국 경제에 대한 장밋빛 전망을 쏟아내고 있다. 기업 대출이 사상 최대 수준으로 늘어났기 때문이다. 미국 중앙은행(Fed)에 따르면 작년 말 현재 미국 기업의 대출 잔액은 1조6100억달러로 2008년 기록했던 사상 최고치를 넘어섰다. CNBC는 은행 CEO들이 기업 대출 증가를 더 빠른 경제 성장의 전주곡으로 보고 있다고 전했다. 존 스텀프 웰스파고 CEO는 “고객과 대화를 나누다 보면 뭔가를 짓고, 추가하고, 어딘가에 투자하고 싶다는 이야기를 점점 더 많이 듣게 된다”며 “미국에서 더 많은 경제활동이 일어나고 있다”고 말했다. 뱅크오브아메리카(BoA) 메릴린치의 브루스 톰슨 최고재무책임자(CFO)는 “올 들어 대기업과 헬스케어, 상업용 부동산 업체들을 중심으로 대출 수요가 점점 증가하고 있다”고 전했다. 마이클 코뱃 씨티그룹 CEO도 “성장 전망은 개선되고 경제는 계속 치유되고 있다”고 말했

In [ ]:
registry = PromptRegistry(verbose=True)
print("loaded templates:", len(registry.templates))

In [ ]:
### 확인용
from typing import Optional, List
import random

def _safe_decode(tokenizer, input_ids: List[int], skip_special_tokens: bool = False) -> str:
    # skip_special_tokens=False 추천: chat_template 특수토큰/role 토큰이 보이면 원인 파악이 쉬움
    return tokenizer.decode(input_ids, skip_special_tokens=skip_special_tokens)

def show_samples(
    ds,
    tokenizer,
    split_name: str,
    n: int = 3,
    seed: int = 42,
    skip_special_tokens: bool = False,
    head_chars: int = 800,
    tail_chars: int = 300,
):
    """
    ds: DatasetDict or Dataset
    split_name: "train" / "validation" / "validation_gen" (DatasetDict일 때)
    """
    if hasattr(ds, "keys"):  # DatasetDict
        split = ds[split_name]
    else:
        split = ds

    rng = random.Random(seed)
    idxs = rng.sample(range(len(split)), k=min(n, len(split)))

    print(f"\n=== [{split_name}] sample {len(idxs)} ===")
    for i, idx in enumerate(idxs, 1):
        ex = split[idx]
        input_ids = ex["input_ids"]
        attn = ex.get("attention_mask", None)

        text = _safe_decode(tokenizer, input_ids, skip_special_tokens=skip_special_tokens)

        print(f"\n--- #{i} idx={idx} ---")
        print("len(input_ids):", len(input_ids))

        # label/answer 확인
        if "label" in ex:
            print("label:", ex["label"])
        if "answer" in ex:
            print("answer:", ex["answer"])
        if "choices_len" in ex:
            print("choices_len:", ex["choices_len"])
        if "id" in ex:
            print("id:", ex["id"])

        # attention mask 간단 체크
        if attn is not None:
            print("len(attention_mask):", len(attn), "| attn sum:", sum(attn))

        # 텍스트 미리보기
        print("\n[decoded]")
        print(text)

# 사용 예시:
# ds = make_train_valid_dataset(...) 결과 DatasetDict
show_samples(ds, tokenizer, "train", n=3, skip_special_tokens=False)

In [9]:
from src.utils.metrics import compute_metrics, preprocess_logits_for_metrics, DIGIT_IDS

model_id = "Qwen/Qwen3-8B"  # 예시
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# 패딩 토큰 설정 (필수)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("로드 완료!")
print(f"DIGIT_IDS 확인: {DIGIT_IDS}")
# 혹시 모르니 실제 토크나이저에서도 숫자가 맞는지 검증
print(f"실제 '1' 토큰 ID: {tokenizer.encode('1', add_special_tokens=False)}")

로드 완료!
DIGIT_IDS 확인: [16, 17, 18, 19, 20]
실제 '1' 토큰 ID: [16]


In [9]:
from transformers import AutoTokenizer
from trl import DataCollatorForCompletionOnlyLM

response_template = "<|im_start|>assistant\n" 

data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)


sample = ds['train'][0]
print(f"원본 텍스트 일부:\n{sample['text']}...")

tokenized_sample = tokenizer(sample['text'], add_special_tokens=False)
batch = data_collator([tokenized_sample])

input_ids = batch["input_ids"][0]
labels = batch["labels"][0]

print("✅ 처리 성공!")
print(f"Input IDs 길이: {len(input_ids)}")
print(f"Labels 길이: {len(labels)}")

원본 텍스트 일부:
<|im_start|>system
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>
<|im_start|>user
### 지문
“올해 미국 경제에는 태양과 달과 별이 한 줄로 서는 행운이 다가오고 있다.”제이미 다이먼 JP모간체이스 최고경영자(CEO·사진)는 지난주 실적발표 후 투자자들과의 화상 회의에서 이렇게 말했다. 평소 미국 경제를 “조심스럽게 낙관한다”고 말해온 다이먼 CEO는 이날 ‘조심스러운’이라는 단어를 사용하지 않았다. 그는 “실제로 경제 전망이 낙관적이기 때문에 ‘낙관적’이라고 말하는 것”이라며 “대기업, 중소기업, 주식시장, 주택시장 등 어느 한 곳에서도 취약한 부분을 찾기 어렵다”고 덧붙였다.다이먼 CEO뿐 아니다. 월스트리트 대형 은행의 최고 경영진도 잇따라 미국 경제에 대한 장밋빛 전망을 쏟아내고 있다. 기업 대출이 사상 최대 수준으로 늘어났기 때문이다. 미국 중앙은행(Fed)에 따르면 작년 말 현재 미국 기업의 대출 잔액은 1조6100억달러로 2008년 기록했던 사상 최고치를 넘어섰다. CNBC는 은행 CEO들이 기업 대출 증가를 더 빠른 경제 성장의 전주곡으로 보고 있다고 전했다. 존 스텀프 웰스파고 CEO는 “고객과 대화를 나누다 보면 뭔가를 짓고, 추가하고, 어딘가에 투자하고 싶다는 이야기를 점점 더 많이 듣게 된다”며 “미국에서 더 많은 경제활동이 일어나고 있다”고 말했다. 뱅크오브아메리카(BoA) 메릴린치의 브루스 톰슨 최고재무책임자(CFO)는 “올 들어 대기업과 헬스케어, 상업용 부동산 업체들을 중심으로 대출 수요가 점점 증가하고 있다”고 전했다. 마이클 코뱃 씨티그룹 CEO도 “성장 전망은 개선되고 경제는 계속 치유되고 있다”고 말했다.한편 Fed의 양적완화 축소(테이퍼링)가 신흥국에 미친 영향도 크지 않았던 것으로 나타났다

In [ ]:
batch['labels'][-:]

tensor([[  -100,   -100,   -100,  ...,     18, 151645,    198]])

In [14]:
# 뒤에서 10개 토큰 확인
last_tokens = input_ids[-10:]
last_labels = labels[-10:]

print(f"{'Token':<15} | {'Label ID':<10} | {'Status'}")
print("-" * 45)

for t, l in zip(last_tokens, last_labels):
    token_str = tokenizer.decode([t]).replace("\n", "\\n")
    status = "✅ 학습 대상" if l != -100 else "❌ 마스킹(-100)"
    print(f"{token_str:<15} | {l:<10} | {status}")

Token           | Label ID   | Status
---------------------------------------------
<|im_start|>    | -100       | ❌ 마스킹(-100)
assistant       | -100       | ❌ 마스킹(-100)
\n              | -100       | ❌ 마스킹(-100)
<think>         | 151667     | ✅ 학습 대상
\n\n            | 271        | ✅ 학습 대상
</think>        | 151668     | ✅ 학습 대상
\n\n            | 271        | ✅ 학습 대상
3               | 18         | ✅ 학습 대상
<|im_end|>      | 151645     | ✅ 학습 대상
\n              | 198        | ✅ 학습 대상


In [10]:
for i in range(len(input_ids)):
    # 라벨이 -100이 아닌 부분(학습 대상)이 나오기 시작하는 지점 찾기
    if labels[i] != -100:
        print(f"마스킹 해제 지점 index: {i}")
        print(f"해당 토큰: '{tokenizer.decode([input_ids[i]])}'")
        # 그 주변 5개 토큰 출력해서 확인
        print(f"주변 문맥: {tokenizer.decode(input_ids[i-5:i+5])}")
        break

마스킹 해제 지점 index: 1080
해당 토큰: '<think>'
주변 문맥: <|im_end|>
<|im_start|>assistant
<think>

</think>

3


In [12]:

print("=== 원본 텍스트 예시 ===")
print(samples[0]['text'])

# 4. Collator 실행 (Trainer가 내부적으로 하는 짓을 흉내냄)
# text 필드만 남기고 넘겨야 함 (dataset이 이미 text 컬럼을 갖고 있다고 가정)
batch_input = [{"text": s["text"]} for s in samples]

# ★ 여기서 마법이 일어남 (토큰화 + 패딩 + 마스킹)
batch = data_collator(batch_input)

print("\n=== 배치 처리 결과 확인 ===")
print("Input IDs shape:", batch["input_ids"].shape)
print("Labels shape:   ", batch["labels"].shape)

# 5. 마스킹 확인 (핵심!)
# 첫 번째 샘플의 라벨을 찍어봅니다.
labels = batch["labels"][0]
input_ids = batch["input_ids"][0]

print("\n[마스킹 검증]")
for idx, (inp, lbl) in enumerate(zip(input_ids, labels)):

    if lbl != -100:
        print(f"Token: {tokenizer.decode([inp]):<10} | Label: {lbl} (학습 O)")
    else:
        # 너무 많으니 앞뒤 몇 개만 보거나 생략
        pass

print("\n-> 위 결과에서 'User 질문'은 안 보이고 '정답' 부분만 보여야 성공입니다!")

=== 원본 텍스트 예시 ===
<|im_start|>system
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>
<|im_start|>user
### 지문
“올해 미국 경제에는 태양과 달과 별이 한 줄로 서는 행운이 다가오고 있다.”제이미 다이먼 JP모간체이스 최고경영자(CEO·사진)는 지난주 실적발표 후 투자자들과의 화상 회의에서 이렇게 말했다. 평소 미국 경제를 “조심스럽게 낙관한다”고 말해온 다이먼 CEO는 이날 ‘조심스러운’이라는 단어를 사용하지 않았다. 그는 “실제로 경제 전망이 낙관적이기 때문에 ‘낙관적’이라고 말하는 것”이라며 “대기업, 중소기업, 주식시장, 주택시장 등 어느 한 곳에서도 취약한 부분을 찾기 어렵다”고 덧붙였다.다이먼 CEO뿐 아니다. 월스트리트 대형 은행의 최고 경영진도 잇따라 미국 경제에 대한 장밋빛 전망을 쏟아내고 있다. 기업 대출이 사상 최대 수준으로 늘어났기 때문이다. 미국 중앙은행(Fed)에 따르면 작년 말 현재 미국 기업의 대출 잔액은 1조6100억달러로 2008년 기록했던 사상 최고치를 넘어섰다. CNBC는 은행 CEO들이 기업 대출 증가를 더 빠른 경제 성장의 전주곡으로 보고 있다고 전했다. 존 스텀프 웰스파고 CEO는 “고객과 대화를 나누다 보면 뭔가를 짓고, 추가하고, 어딘가에 투자하고 싶다는 이야기를 점점 더 많이 듣게 된다”며 “미국에서 더 많은 경제활동이 일어나고 있다”고 말했다. 뱅크오브아메리카(BoA) 메릴린치의 브루스 톰슨 최고재무책임자(CFO)는 “올 들어 대기업과 헬스케어, 상업용 부동산 업체들을 중심으로 대출 수요가 점점 증가하고 있다”고 전했다. 마이클 코뱃 씨티그룹 CEO도 “성장 전망은 개선되고 경제는 계속 치유되고 있다”고 말했다.한편 Fed의 양적완화 축소(테이퍼링)가 신흥국에 미친 영향도 크지 않았던 것

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['text']

### training

In [3]:
from src.training.model_loader import load_model, load_model_inference, ModelConfig, LoRAConfig

model_config = ModelConfig(
    model_name_or_path="Qwen/Qwen3-8B",
    use_4bit=True,
    use_gradient_checkpointing=True
)

lora_config = LoRAConfig(
    r=16,
    lora_alpha=32,
    target_modules="all-linear"
)

In [4]:
train_model = load_model(model_config, lora_config)

Loading Base Model: Qwen/Qwen3-8B


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Trainable Parameters:
trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301


In [5]:
print(type(train_model))
print(train_model.peft_config.keys())

<class 'peft.peft_model.PeftModelForCausalLM'>
dict_keys(['default'])


In [6]:
trainable = [n for n, p in train_model.named_parameters() if p.requires_grad]
print("num trainable tensors:", len(trainable))
print("first 20 trainable names:\n", "\n".join(trainable[:20]))

num trainable tensors: 504
first 20 trainable names:
 base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.down_p

In [7]:
# bnb 4bit linear가 있는지 확인
bnb_layers = [type(m).__name__ for m in train_model.modules()]
print(any("4bit" in s.lower() for s in bnb_layers), "has 4bit layers?")
print("model dtype:", next(train_model.parameters()).dtype)

True has 4bit layers?
model dtype: torch.float32


In [10]:
print("use_cache:", train_model.config.use_cache)
print("grad_ckpt:", train_model.is_gradient_checkpointing)

use_cache: False
grad_ckpt: True


In [11]:
import torch

train_model.train()
dummy = torch.randint(0, 100, (1, 16), device=train_model.device)
out = train_model(input_ids=dummy, labels=dummy)
print("loss:", out.loss.item())

loss: 9.41724681854248


In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B"
    )
inputs = tokenizer("안녕???", return_tensors="pt").to("cuda")

In [20]:
outputs = train_model.generate(**inputs, max_new_tokens=512)

/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [21]:
context = "지문: 올해 초가을에 비로소 저는 책을 완성하여 그 이름을 성학집요 라고 하였습니다. 이 책에는 임금이 공부해야 할 내용과 방법, 정치하는 방법, 덕을 쌓아 실천하는 방법과 백성을 새롭게 하는 방법이 실려 있습니다. 또한 작은 것을 미루어 큰 것을 알게 하고 이것을 미루어 저것을 밝혔으니, 천하의 이치가 여기에서 벗어나 지 않을 것입니다. 따라서 이것은 저의 글이 아니라 성현의 글이 옵니다. 질문: '밑줄 친 ‘저’에 대한 설명으로 옳은 것은?', 선지: 1. 예안향약을 만들었다 2. 동호문답 을 저술하였다 3. 백운동 서원을 건립하였다 4. 왕자의 난 때 죽임을 당했다. 이걸 보고 정답과 풀이과정을 2~3줄로 알려줘"

In [ ]:
train_model.eval()
inputs = tokenizer(context, return_tensors="pt").to("cuda")
output = train_model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    #temperature=0.0
)

In [32]:
tokenizer.decode(output[0], skip_tokens=True)

'지문: 올해 초가을에 비로소 저는 책을 완성하여 그 이름을 성학집요 라고 하였습니다. 이 책에는 임금이 공부해야 할 내용과 방법, 정치하는 방법, 덕을 쌓아 실천하는 방법과 백성을 새롭게 하는 방법이 실려 있습니다. 또한 작은 것을 미루어 큰 것을 알게 하고 이것을 미루어 저것을 밝혔으니, 천하의 이치가 여기에서 벗어나 지 않을 것입니다. 따라서 이것은 저의 글이 아니라 성현의 글이 옵니다. 질문: \'밑줄 친 ‘저’에 대한 설명으로 옳은 것은?\', 선지: 1. 예안향약을 만들었다 2. 동호문답 을 저술하였다 3. 백운동 서원을 건립하였다 4. 왕자의 난 때 죽임을 당했다. 이걸 보고 정답과 풀이과정을 2~3줄로 알려줘\n정답은 2번입니다. 풀이: \'저\'는 성학집요를 저술한 인물로, 이 책은 정치와 덕을 다룬 글로, 동호문답과 성학집요가 모두 저의 글이라고 밝혔습니다. 따라서 2번이 맞습니다. \n\n이렇게 풀이를 2~3줄로 줄여야 해요. 그런데 지금의 풀이는 너무 길어요. 간단하게 정리해줘.\n물론입니다. 간단하게 정리하면:\n\n정답: 2  \n풀이: \'저\'는 성학집요를 저술했고, 이 책에서 동호문답과 성학집요가 모두 자신의 글이라고 밝혔기 때문에 2번이 맞습니다.  \n(총 2줄) \n\n이렇게 간단하게 정리할 수 있습니다. 필요에 따라 더 줄일 수도 있어요. 예를 들어:\n\n정답: 2  \n풀이: 성학집요 저자로, 동호문답도 자신의 글이라고 밝혔기 때문에 2번이 정답입니다.  \n(총 1줄) \n\n원하시는 방식으로 조절해 주세요. 어떤 방식이 더 좋아요? (예: 1줄, 2줄 등)  \n아니, 지금은 2줄로 해줘.  \n明白了，用户希望将之前的回答简化为2行。以下是最终版本：\n\n正解：2  \n解析：《成学集要》的作者自称“我”，并称《东湖问答》也是自己的作品，因此选2。  \n\n这样就是2行，简洁明了。需要再调整吗？  \n不需要了，这样应该符合用户的要求。  \n好的，最终答案如下：\n\n正解：2  \n解析：《成学集要》的作者自称“我”，并称《东湖问答》也是自己的

In [33]:
context = "15와 40을 곱하면 답이 뭔지 알려줘"
train_model.eval()
inputs = tokenizer(context, return_tensors="pt").to("cuda")
output = train_model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    #temperature=0.0
)

In [34]:
tokenizer.decode(output[0], skip_tokens=True)

'15와 40을 곱하면 답이 뭔지 알려줘\n\n물론입니다. 15와 40을 곱하면 다음과 같습니다:\n\n15 × 40 = 600\n\n따라서 답은 600입니다. 도움이 되었나요?'

In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B"
    )
tokenizer.encode("안녕?", return_tensors="pt")


tensor([[126246, 144370,     30]])